In [5]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
from bq_helper import BigQueryHelper
%load_ext google.cloud.bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="My Project-bbdce7b1712b.json"

bq_assistant = BigQueryHelper("bigquery-public-data", "epa_historical_air_quality")

query="""
SELECT * FROM `bigquery-public-data.san_francisco.sfpd_incidents` 
where EXTRACT(YEAR FROM TIMESTAMP)=2016
"""

df = bq_assistant.query_to_pandas(query)


The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [6]:
df

,unique_key,category,descript,dayofweek,pddistrict,resolution,address,longitude,latitude,location,pdid,timestamp
0,166018573,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Sunday,,NONE,100 Block of VELASCO AV,-122.413352,37.708202,"(37.70820245849022, -122.4133519852842)",16601857306244,2016-01-17 23:54:00+00:00
1,160731121,BURGLARY,"BURGLARY OF RESIDENCE, UNLAWFUL ENTRY",Thursday,PARK,NONE,500 Block of DUBOCE AV,-122.433013,37.769159,"(37.7691592647351, -122.43301268519)",16073112105043,2016-09-08 19:00:00+00:00
2,160592826,DRIVING UNDER THE INFLUENCE,DRIVING WHILE UNDER THE INFLUENCE OF ALCOHOL,Saturday,PARK,"ARREST, BOOKED",PORTOLA DR / WOODSIDE AV,-122.451623,37.745533,"(37.745533043349, -122.4516228695718)",16059282665050,2016-07-23 02:51:00+00:00
3,160670331,ASSAULT,BATTERY WITH SERIOUS INJURIES,Thursday,PARK,NONE,PORTOLA DR / WOODSIDE AV,-122.451623,37.745533,"(37.745533043349, -122.4516228695718)",16067033104136,2016-08-18 08:00:00+00:00
4,160592826,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Saturday,PARK,"ARREST, BOOKED",PORTOLA DR / WOODSIDE AV,-122.451623,37.745533,"(37.745533043349, -122.4516228695718)",16059282665010,2016-07-23 02:51:00+00:00
5,160952054,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Wednesday,PARK,NONE,PORTOLA DR / WOODSIDE AV,-122.451623,37.745533,"(37.745533043349, -122.4516228695718)",16095205428160,2016-11-16 13:00:00+00:00
6,160044388,NON-CRIMINAL,AIDED CASE,Saturday,PARK,NONE,100 Block of BRODERICK ST,-122.438874,37.771524,"(37.77152366100431, -122.43887424471)",16004438851040,2016-01-16 11:13:00+00:00
7,166294494,LARCENY/THEFT,GRAND THEFT OF PROPERTY,Tuesday,PARK,NONE,100 Block of BRODERICK ST,-122.438874,37.771524,"(37.77152366100431, -122.43887424471)",16629449406374,2016-12-13 08:05:00+00:00
8,160256668,LARCENY/THEFT,PETTY THEFT FROM A BUILDING,Monday,PARK,NONE,100 Block of BRODERICK ST,-122.438874,37.771524,"(37.77152366100431, -122.43887424471)",16025666806302,2016-03-21 10:11:00+00:00
9,160820451,FRAUD,"CREDIT CARD, THEFT BY USE OF",Friday,PARK,NONE,100 Block of BRODERICK ST,-122.438874,37.771524,"(37.77152366100431, -122.43887424471)",16082045109320,2016-09-30 12:00:00+00:00


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
unique_key    10000 non-null int64
category      10000 non-null object
descript      10000 non-null object
dayofweek     10000 non-null object
pddistrict    9999 non-null object
resolution    10000 non-null object
address       10000 non-null object
longitude     10000 non-null float64
latitude      10000 non-null float64
location      10000 non-null object
pdid          10000 non-null int64
timestamp     10000 non-null object
dtypes: float64(2), int64(2), object(8)
memory usage: 937.6+ KB


In [10]:
df.nunique()

unique_key     9442
category         36
descript        502
dayofweek         7
pddistrict        1
resolution       16
address        1145
longitude      1832
latitude       1819
location       1838
pdid          10000
timestamp      9336
dtype: int64

In [12]:
set(df['resolution'])

{'ARREST, BOOKED',
 'ARREST, CITED',
 'CLEARED-CONTACT JUVENILE FOR MORE INFO',
 'COMPLAINANT REFUSES TO PROSECUTE',
 'DISTRICT ATTORNEY REFUSES TO PROSECUTE',
 'EXCEPTIONAL CLEARANCE',
 'JUVENILE ADMONISHED',
 'JUVENILE BOOKED',
 'JUVENILE CITED',
 'JUVENILE DIVERTED',
 'LOCATED',
 'NONE',
 'NOT PROSECUTED',
 'PROSECUTED BY OUTSIDE AGENCY',
 'PSYCHOPATHIC CASE',
 'UNFOUNDED'}

In [13]:
pip install --upgrade google-cloud-bigquery[pandas]



The following command must be run outside of the IPython shell:

    $ pip install --upgrade google-cloud-bigquery[pandas]

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [14]:
%pip install --upgrade google-cloud-bigquery[pandas]


The following command must be run outside of the IPython shell:

    $ pip install --upgrade google-cloud-bigquery[pandas]

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/
